In [4]:
import os
import sys
sys.dont_write_bytecode = True
os.environ["NUMBA_DISABLE_JIT"] = "1"
import plotly.express as px
import plotly.graph_objects as go


import pandas as pd
import numpy as np
from quantfreedom.utils.caching import clear_cache
from quantfreedom.backtester.base.base import  plot_trades_all_info
from quantfreedom.backtester.nb.simulate import simulate_up_to_6
from quantfreedom.backtester.enums.enums import (
    LeverageMode,
    SizeType,
    OrderType,
    SL_BE_or_Trail_BasedOn,
)
from quantfreedom.backtester.indicators.talib_ind import from_talib
from quantfreedom.backtester.evaluators.evaluators import eval_is_below

pd.options.display.float_format = '{:,.2f}'.format

prices = pd.read_csv(
    'E:/Coding/backtesters/QuantFreedom/tests/data/30min.csv', index_col='time')

rsi_ind = from_talib(
    func_name='rsi',
    df_prices=prices,
    cart_product=False,
    combos=False,
    timeperiod=15,
)
rsi_eval = eval_is_below(
    rsi_ind,
    np.arange(40, 61, 10),
)


In [5]:
final_array, order_records = simulate_up_to_6(
    open_prices=prices.open.values,
    high_prices=prices.high.values,
    low_prices=prices.low.values,
    close_prices=prices.close.values,
    entries=rsi_eval.values,
    equity=1000.,
    fee_pct=.06,
    mmr=.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=4,
    risk_rewards=[3,5,6],
    size_pct=1.,
    # sl_pcts=[2,3,4],
    tsl_true_or_false=True,
    tsl_pcts_init=np.arange(2,5,1),
    tsl_based_on=SL_BE_or_Trail_BasedOn.low_price,
    tsl_trail_by_pct=np.arange(1,4,1),
    tsl_when_pct_from_avg_entry=np.arange(1,4,1),
)


In [22]:
headerColor = 'grey'
rowEvenColor = 'lightgrey'
rowOddColor = 'white'

fig = go.Figure(
    data=[
        go.Table(
            header=dict(
                values=(order_records.dtype.names),
                line_color="darkslategray",
                fill_color=headerColor,
                align=["left", "center"],
                font=dict(color="white", size=12),
            ),
            cells=dict(
                values=pd.DataFrame(order_records[:20]).T.values,
                line_color="darkslategray",
                # 2-D list of colors for alternating rows
                # fill_color=[
                #     [
                #         rowOddColor,
                #         rowEvenColor,
                #         rowOddColor,
                #         rowEvenColor,
                #         rowOddColor,
                #     ]
                #     * 5
                # ],
                align=["left", "center"],
                font=dict(color="darkslategray", size=11),
            ),
        )
    ]
)

fig.show()


In [18]:
pd.DataFrame(order_records).values

array([[0.00000000e+00, 0.00000000e+00, 3.10000000e+01, ...,
                   nan, 1.06330000e+04, 1.15531113e+04],
       [1.00000000e+00, 0.00000000e+00, 3.20000000e+01, ...,
                   nan, 1.05713951e+04, 1.14861755e+04],
       [2.00000000e+00, 0.00000000e+00, 3.20000000e+01, ...,
                   nan, 1.05713951e+04, 1.14861755e+04],
       ...,
       [3.68000000e+03, 2.00000000e+00, 6.94300000e+03, ...,
                   nan, 5.03827606e+04, 6.55188462e+04],
       [3.68100000e+03, 2.00000000e+00, 6.94400000e+03, ...,
                   nan, 5.05559067e+04, 6.57440093e+04],
       [3.68200000e+03, 2.00000000e+00, 6.95600000e+03, ...,
                   nan, 5.26724550e+04, 6.57440093e+04]])

In [8]:
order_records.T

array([(   0, 0,   31, 471.96526336, 10850.        , 10850.        ,        nan,  0.,  nan, 1000.        , nan, 10633.        , 11553.11126676),
       (   1, 0,   32, 471.96526336, 10725.        , 10787.13789108,        nan,  0.,  nan, 1000.        , nan, 10571.39513326, 11486.17551203),
       (   2, 0,   32, 943.93052671, 10571.39513326, 10787.13789108, 1.12138947,  6., -20.,  980.        , nan, 10571.39513326, 11486.17551203),
       ...,
       (3680, 2, 6943, 658.17390615, 52890.        , 52482.04226899,        nan,  0.,  nan, 2710.09687595, nan, 50382.76057823, 65518.8462369 ),
       (3681, 2, 6944, 658.17390615, 53211.        , 52662.40284145,        nan,  0.,  nan, 2710.09687595, nan, 50555.90672779, 65744.00928513),
       (3682, 2, 6956,          nan,            nan, 52662.40284145,        nan, 12.,  nan, 2710.09687595, nan, 52672.455     , 65744.00928513)],
      dtype={'names': ['order_id', 'settings_id', 'bar', 'size_value', 'price', 'avg_entry', 'fees_paid', 'order_type